In [76]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
import time
import random
import requests
import os
import glob
import re
import csv


class MOST_list_text_crawl():

    def __init__(self, driver_path, download_path, crawler_year):
        self.url_to_crawl = 'https://wsts.nstc.gov.tw/STSWeb/Award/AwardMultiQuery.aspx'
        self.driver_path = driver_path
        self.regi_path = regi_path
        self.SaveFile_Path = SaveFile_Path
        self.SaveFile_Name = SaveFile_Name
        self.proj = proj
        self.proj_aut = proj_aut
        self.agency = agency
        self.proj_year = proj_year

    def start(self):
        self.browser = webdriver.Chrome(self.driver_path)
        self.browser.maximize_window()

    def get_into_page(self, url):
        self.browser.get(url)
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到出現 「科技部補助研究計畫」
            EC.presence_of_element_located((By.ID, "dtlItem_btnItem_0")))
        self.browser.find_element(By.ID, 'dtlItem_btnItem_0').click()

    def search_work(self, unit):
        Select(
            self.browser.find_element(
                By.XPATH, '//*[@id="wUctlAwardQueryPage_repQuery_ddlYRst_0"]')
        ).select_by_value(str(proj_year[self.i]))
        Select(
            self.browser.find_element(
                By.XPATH, '//*[@id="wUctlAwardQueryPage_repQuery_ddlYRend_0"]')
        ).select_by_value(str(proj_year[self.i]))
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到出現 "搜尋框"
            EC.presence_of_element_located(
                (By.ID, "wUctlAwardQueryPage_repQuery_txtT_4")))
        self.browser.find_element(
            By.ID, "wUctlAwardQueryPage_repQuery_txtT_4").send_keys(
                proj.iloc[self.i][:unit])
        self.browser.find_element(By.ID,
                                  "wUctlAwardQueryPage_btnQuery").click()
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到出現 "搜尋框"
            EC.presence_of_element_located(
                (By.ID, "wUctlAwardQueryPage_repQuery_txtT_4")))

    def download_file(self, soup):
        result_list = []
        title = soup.find("span",
                          string="計畫名稱：").find_next_sibling("span").text
        starend = soup.find(
            "span",
            string="執行起迄：").find_next_sibling("span").text.replace('/', '-')
        ch_keyword = soup.find("span",
                               string="中文關鍵字：").find_next_sibling("span").text
        eng_keyword = soup.find("span",
                                string="英文關鍵字：").find_next_sibling("span").text
        ch_abst = soup.find("span",
                            string="中文摘要：").find_next_sibling("span").text
        eng_abst = soup.find("span",
                             string="英文摘要：").find_next_sibling("span").text
        error = ''
        result_list.append([
            proj_aut.iloc[self.i], starend, agency.iloc[self.i], title,
            ch_keyword, eng_keyword, ch_abst, eng_abst, error
        ])

        return result_list

    def roll_page(self):  #翻頁並進入search_wor和download_file的步驟
        res_all = pd.DataFrame()
        proj_all = len(projTT)
        lst1 = list(range(0, proj_all))
        res_process = pd.DataFrame()
        self.error_list = []
        for self.i in lst1:
            soup = BeautifulSoup(self.browser.page_source, 'lxml')  #擷取每頁頁面
            self.search_work(500)
            try:
                soup = BeautifulSoup(self.browser.page_source, 'lxml')  #擷取每頁頁面
                result_list = self.download_file(soup)
                self.browser.find_element(
                    By.ID, "wUctlAwardQueryPage_repQuery_txtT_4").clear()
                result_df = pd.DataFrame(result_list)
                res_process = res_process.append(result_df)
            except:
                try:
                    self.browser.find_element(
                        By.ID, "wUctlAwardQueryPage_repQuery_txtT_4").clear()
                    self.search_work(20)
                    soup = BeautifulSoup(self.browser.page_source,
                                         'lxml')  #擷取每頁頁面
                    result_list = self.download_file(soup)
                    self.browser.find_element(
                        By.ID, "wUctlAwardQueryPage_repQuery_txtT_4").clear()
                    result_df = pd.DataFrame(result_list)
                    res_process = res_process.append(result_df)
                except:
                    starend = ''
                    title = ''
                    agency = ''
                    ch_keyword = ''
                    eng_keyword = ''
                    ch_abst = ''
                    eng_abst = ''
                    error = '程式在MOST查無資料，請手動查詢。'
                    self.error_list.append([
                        proj_aut.iloc[self.i], starend, agency, proj.iloc[self.i],
                        ch_keyword, eng_keyword, ch_abst, eng_abst, error
                    ])
                    result_df = pd.DataFrame(self.error_list)
                    res_process = res_process.append(result_df)
                    self.browser.find_element(
                        By.ID, "wUctlAwardQueryPage_repQuery_txtT_4").clear()
                    
        res_all = res_all.append(res_process)
        res_all.columns = [
            '計畫主持人', '執行起迄', '機關名稱', '計畫名稱', '中文關鍵字', '英文關鍵字', '中文摘要', '英文摘要',
            '缺失情況'
        ]
        res_all = res_all.drop_duplicates(subset='計畫名稱',
                              keep='first',
                              inplace=False,
                              ignore_index=False)
        try:
            res_all.to_excel(SaveFile_Path + '\\' + SaveFile_Name + "_" + "_" +
                             time.strftime("%m%d") + "_" +
                             time.strftime("%H%M") + ".xlsx",
                             encoding="utf_8_sig",
                             index=False)  #儲存結果
            print("存檔成功")  #最終檔名會加上 1.月份 2.日期 3.時間(時分)
            print('原始清單共有' + str(len(proj)) + '筆資料')
            print('MOST資料檔共有' + str(len(res_all)) + '筆資料')
        except:
            print("存檔失敗，請檢察結果檔是否正在開啟，或處於無法被寫入的狀態。")
        self.error_check()

    def error_check(self):
        try:
            errof_df = pd.DataFrame(self.error_list)
            error_check = '程式在MOST查無資料，請手動查詢。' in (errof_df.iloc[:,8].values)
            if error_check == True:
                print('有資料缺失')
                print('共有' + str(len(self.error_list)) + '筆缺失資料')
        except:
            print('無資料缺失')


    def close_driver(self):
        self.browser.quit()
        print('成功爬取MOST清冊')

    def step(self):
        self.start()
        self.get_into_page(self.url_to_crawl)
        print('正在爬取MOST清冊')
        self.roll_page()
        self.close_driver()


if __name__ == '__main__':
    driver_path = r'C:\Users\user\Desktop\py\Driver\chromedriver.exe'  #chromedriver位置
    regi_path = r"C:\Users\user\Desktop\py\merge\108調查計畫名冊_全司1081122.xlsx"  #科技部名冊路徑 ★請注意前面必須有小r★
    SaveFile_Path = r'C:\Users\user\Desktop\py\TEST'  #要儲存檔案的路徑
    SaveFile_Name = r'MOST_list_text_crawl'  #儲存的檔名，可以不用更動
    projTT = pd.read_excel(regi_path)
    proj = projTT['計畫名稱']
    proj_aut = projTT['計畫主持人']
    agency = projTT['機關名稱']
    proj_year = projTT['補'].astype('Int64')
    go = MOST_list_text_crawl(driver_path, SaveFile_Path, regi_path)
    go.step()

C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.browser = webdriver.Chrome(self.driver_path)


正在爬取MOST清冊


C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppDat

C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppDat

C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppDat

C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppDat

C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppDat

C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppDat

C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_9412\240734013.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppDat

存檔成功
原始清單共有216筆資料
MOST資料檔共有216筆資料
有資料缺失
共有3筆缺失資料
成功爬取MOST清冊


In [59]:
regi_path = r"C:\Users\user\Desktop\py\TEST\MOST_list_text_crawl__0816_1603.xlsx"  

projTT = pd.read_excel(regi_path)
projTT


error_check = '程式查無資料，請手動查詢。' in projTT.iloc[:,8].values
error_check

if error_check == True:
    print('有資料缺失')
else:
    print('無資料缺失')
    
    



無資料缺失


C:\Users\user\AppData\Local\Temp\ipykernel_9412\3590229825.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  error_check = '程式查無資料，請手動查詢。' in projTT.iloc[:,8].values


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
187   NaN
188   NaN
189   NaN
190   NaN
191   NaN
Name: 缺失情況, Length: 192, dtype: float64

In [60]:
projTT.iloc[:,8]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [19]:
error_check

True